In [ ]:
#To set up your workspace, use your specified filepath
#Be sure to change all directory and file names to your own

In [ ]:
#Permit Google Colab to access files in google drive
#Can either (1) mount using the following line of code or (2) set manually in left hand 'Files' panel if using Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install packages 
!pip install geopandas
!pip install rasterio
!pip install rasterstats
!pip install rioxarray

In [ ]:
#Import required libraries
##Must be run each time session is started
import glob 
import os 
import ogr
import gdal, gdalconst 

import rasterio as rio
from rasterio.plot import show
import rasterstats 
import rioxarray
import xarray
from shapely.geometry import mapping

import numpy as np 
import geopandas as gpd 
import matplotlib.pyplot as plt

# Pre-2016 Carbon Stock

In [ ]:
# STEP 1 - Reproject layers to same resolution and projection
filepath = '/content/drive/my_file_path/AGB_Years' ## file path of above ground biomass yearly files, filepath can be copied directly from google drive
data = glob.glob(os.path.join(filepath, '*.tif')) # opens folder where files are saved

for i, data_path in enumerate(data):
  raster = gdal.Open(data_path, i) # open each raster 
  name = os.path.splitext(os.path.split(data_path)[1])[0] # retain names of input file
  output_path = f'/content/drive/my_file_path/Processing/pre_{name}.tif' ## define outfile 
  gdal.Warp(output_path,raster,dstSRS='EPSG:26918', xRes = '30', yRes = '30') # reproject 

raster = None # close files 

In [ ]:
# STEP 2 - Clip data layers to project boundary 
filepath = '/content/drive/Processing' ##file path of data where you saved the reprojected layers (Refer to output_path in step 2)
data = glob.glob(os.path.join(filepath, '*.tif'))
shp = '/content/drive/my_file_path/Project_Boundary' ## file path of project boundary (Project Boundary shapefile)

for i, data_path in enumerate(data):
  raster = gdal.Open(data_path, i) 
  name = os.path.splitext(os.path.split(data_path)[1])[0][4:13] # get original band names 
  output_path = f'/content/drive/my_file_path/Extraction/extract_{name}.tif' ## define outfile to save new layers
  gdal.Warp(output_path, raster, cutlineDSName= shp,cropToCutline=True) # clip 

raster = None # close files

In [ ]:
# STEP 3 - Extract AGB values from raster

filepath = '/content/drive//my_file_path/Extraction' ## file path of data where you saved clipped layers (step 3)
data = glob.glob(os.path.join(filepath, '*.tif'))

for i, data_path in enumerate(data):
  raster = rioxarray.open_rasterio(data_path, i)
  raster = raster.where(raster >= -100) # mask out no data values 
  year = os.path.splitext(os.path.split(data_path)[1])[0][13:17] # get associated years
  expression = (raster*900).sum()/1000 #unit conversion to Mg C
  print(year, ':', expression.values) ## sort printed results sequentially in excel or similar software

# can sort sequentially in excel 

In [ ]:
raster = None # close files 